In [17]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
text = convert_pdf_to_txt('citycouncilminute.pdf')
print text

In [ ]:
text = convert_pdf_to_txt('citycouncilminute2017.pdf')
print text

In [9]:
from bs4 import BeautifulSoup


In [11]:
BASE_URL = "http://knoxvillecitytn.iqm2.com/Citizens/"
BASE_MINUTES = "http://knoxvillecitytn.iqm2.com/Citizens/calendar.aspx?From=1/1/1900&To=12/31/9999"
import requests
res = requests.get(BASE_MINUTES)
minuteslinks_html = res.content

In [20]:
from tempfile import NamedTemporaryFile
soup = BeautifulSoup(minuteslinks_html)
downloads = {}
for a in soup.find_all('a', href=True, text="Minutes"):
    if not a['href']:
        continue
    minute_url = a['href'].replace('FileView.aspx', 'FileOpen.aspx')
    res = requests.get(BASE_URL + minute_url)
    with NamedTemporaryFile(suffix='pdf') as fin:
        fin.write(res.content)
        fin.seek(0)
        fin.flush()
        text = convert_pdf_to_txt(fin.name)
        downloads[minute_url] = text
    print minute_url
        

FileOpen.aspx?Type=15&ID=1014&Inline=True
FileOpen.aspx?Type=15&ID=1015&Inline=True
FileOpen.aspx?Type=15&ID=1016&Inline=True
FileOpen.aspx?Type=15&ID=1017&Inline=True
FileOpen.aspx?Type=15&ID=1018&Inline=True
FileOpen.aspx?Type=15&ID=1019&Inline=True
FileOpen.aspx?Type=15&ID=1020&Inline=True
FileOpen.aspx?Type=15&ID=1021&Inline=True
FileOpen.aspx?Type=15&ID=1022&Inline=True
FileOpen.aspx?Type=15&ID=1023&Inline=True
FileOpen.aspx?Type=15&ID=1024
FileOpen.aspx?Type=15&ID=1025&Inline=True
FileOpen.aspx?Type=15&ID=1026&Inline=True
FileOpen.aspx?Type=15&ID=1027&Inline=True
FileOpen.aspx?Type=15&ID=1028&Inline=True
FileOpen.aspx?Type=15&ID=1029&Inline=True
FileOpen.aspx?Type=15&ID=1030&Inline=True
FileOpen.aspx?Type=15&ID=1031&Inline=True
FileOpen.aspx?Type=15&ID=1032&Inline=True
FileOpen.aspx?Type=15&ID=1033&Inline=True
FileOpen.aspx?Type=15&ID=1034
FileOpen.aspx?Type=15&ID=1036
FileOpen.aspx?Type=15&ID=1037&Inline=True
FileOpen.aspx?Type=15&ID=1038
FileOpen.aspx?Type=15&ID=1039
FileOpen.a

Use above downloaded data to analyze
------

In [1]:
import pickle
# with open('councilminutes/rawtext.pickle', 'wb') as fout:
#     pickle.dump(downloads, fout)
with open('councilminutes/rawtext.pickle', 'rb') as fin:
    downloads = pickle.load(fin)

In [24]:
import re
from dateutil.parser import parse as dateparse
minute_data = {}
for url, data in downloads.iteritems():
    try:
        datestring = re.findall("\w+, \w+ \d+, \d+", data)[0]
    except:
        print "Failed on", url
        continue
    parsed_date = dateparse(datestring)
    minute_data[str(parsed_date)] = {
        'url': url,
        'date': parsed_date,
        'data': data,
        'isminutes': True
        }
    

Failed on FileOpen.aspx?Type=15&ID=1191&Inline=True
Failed on FileOpen.aspx?Type=15&ID=1192&Inline=True


In [34]:
for date, data in minute_data.iteritems():
    # get the start time
    test_data =  data['data']
    result = re.search('CALL TO ORDER((.|\n)*)INVOCATION AND PLEDGE OF ALLEGIANCE TO THE FLAG', test_data)
    try:
        text = result.group(1)
    except:
        data['isminutes'] = False
    time = re.findall('\d?\d:\d\d\s?[PM|p\.m\.|pm|p\sm]+', text)[0]
    data['starttime'] = dateparse(time).time()
    
    result = re.search('ADJOURNMENT((.|\n)*)City Council Minutes', test_data)
    try:
        text = result.group(1)
    except:
        data['isminutes'] = False

    time = re.findall('\d?\d:\d\d\s?[PM|p\.m\.|pm|p\sm]+', text)[0]
    data['endtime'] = dateparse(time).time()


FileOpen.aspx?Type=15&ID=1023&Inline=True
20:54:00
FileOpen.aspx?Type=15&ID=1037&Inline=True
19:51:00
FileOpen.aspx?Type=15&ID=1115
19:48:00
FileOpen.aspx?Type=15&ID=1049
20:02:00
FileOpen.aspx?Type=15&ID=1105
21:36:00
FileOpen.aspx?Type=15&ID=1088
20:58:00
FileOpen.aspx?Type=15&ID=1190&Inline=True
18:59:00
!!!
FileOpen.aspx?Type=15&ID=1163
18:59:00
FileOpen.aspx?Type=15&ID=1047
19:28:00
FileOpen.aspx?Type=15&ID=1143
20:18:00
FileOpen.aspx?Type=15&ID=1180&Inline=True
18:56:00
FileOpen.aspx?Type=15&ID=1042
20:44:00
FileOpen.aspx?Type=15&ID=1063&Inline=True
19:34:00
FileOpen.aspx?Type=15&ID=1028&Inline=True
20:24:00
FileOpen.aspx?Type=15&ID=1036
19:57:00
FileOpen.aspx?Type=15&ID=1108
20:36:00
FileOpen.aspx?Type=15&ID=1118
19:40:00
FileOpen.aspx?Type=15&ID=1029&Inline=True
20:25:00
FileOpen.aspx?Type=15&ID=1158
19:54:00
FileOpen.aspx?Type=15&ID=1117
20:08:00
FileOpen.aspx?Type=15&ID=1184&Inline=True
18:38:00
FileOpen.aspx?Type=15&ID=1025&Inline=True
19:56:00
FileOpen.aspx?Type=15&ID=1186&

In [94]:
finalresolutions = []
numfails = 0
success = 0
for date, data in minute_data.iteritems():
    if not data['isminutes']:
        continue
    print date
    text = data['data']
    result = re.search('ELECTIONS AND CONFIRMATIONS((.|\n)*)ADJOURNMENT', text)
    textsection = re.sub("City\sCouncil\sMinutes\n\n(.*)+\n\nPage\s\d+\n", "", result.group(1))
    resolutions = re.split("\n+Ordinance|Resolution\sNo\.\s", textsection)
    print len(resolutions)
    for resolutionbase in resolutions[1:]:
        try:
            resolutioncode = resolutionbase.split("\n")[0]
            thetext = ""
            for l in resolutionbase.split("\n")[4:]:
                if l.find("RESULT:") != -1:
                    break
                thetext += l
            resultbase = re.split("\n\nRESULT\:", resolutionbase)[1]
            labels = resultbase.split('\n\n')[0].split('\n')
            labels[0] = "RESULT:"
            theresults = resultbase.split('\n\n')[1].split('\n')
            results = {
                'result': None,
                'mover': None,
                'seconder': None,
                'ayes': None,
                'nays': None,
                'abstains': None,
            }
            
            for l, r in zip(labels, theresults):
                if l.find("RESULT") != -1:
                    results['result'] = r
                elif l.find("MOVER") != -1:
                    results['mover'] = r
                elif l.find("SECONDER") != -1:
                    results['seconder'] = r
                elif l.find("AYES") != -1:
                    results['ayes'] = r
                elif l.find("NAYS") != -1:
                    results['nays'] = r
                elif l.find("ABSTAIN") != -1:
                    results['abstain'] = r
        except:
            print "failed on 1"
            numfails += 1
        else:
            success +=1
            finaldict = {
                    'code': resolutioncode,
                    'text': thetext,
                    'date': date,
                    }
            finaldict.update(results)
            finalresolutions.append(finaldict)
print "fails", numfails
print "success", success
    
    

2012-05-29 00:00:00
23
2012-11-27 00:00:00
17
2015-05-12 00:00:00
4
2013-04-30 00:00:00
19
2015-01-06 00:00:00
13
2014-05-13 00:00:00
20
2013-04-02 00:00:00
22
2013-02-05 00:00:00
10
2013-07-09 00:00:00
9
2012-08-07 00:00:00
19
2012-11-13 00:00:00
20
failed on 1
failed on 1
failed on 1
failed on 1
2015-02-17 00:00:00
20
2015-06-23 00:00:00
18
failed on 1
2012-08-21 00:00:00
24
2015-06-09 00:00:00
12
2012-06-26 00:00:00
20
failed on 1
2015-07-07 00:00:00
14
2014-01-21 00:00:00
20
2015-09-30 00:00:00
22
failed on 1
2012-03-06 00:00:00
12
2015-04-28 00:00:00
15
2015-07-21 00:00:00
25
failed on 1
failed on 1
2013-05-14 00:00:00
17
2014-04-01 00:00:00
37
2013-10-15 00:00:00
14
2012-06-12 00:00:00
13
2013-06-11 00:00:00
13
2014-06-10 00:00:00
18
2012-10-02 00:00:00
16
2013-01-22 00:00:00
32
failed on 1
2015-09-15 00:00:00
18
failed on 1
2015-09-01 00:00:00
13
failed on 1
2013-11-26 00:00:00
16
2013-01-08 00:00:00
21
2014-04-29 00:00:00
24
2014-07-22 00:00:00
25
2012-10-30 00:00:00
25
2014-10

In [96]:
with open('councilminutes/finaldecisions.pickle', 'wb') as fout:
    pickle.dump(finalresolutions, fout)